**worker**

## Live Gezichtsherkenning

In [1]:
#RTSP_URL = "rtsp://wellantmilieu@gmail.com:q8oEZQC!W$8d@192.168.178.22:554/stream2"

In [1]:
import cv2
import os
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from threading import Thread, Lock
import time

In [2]:
# Hoofdmap waar alle datasets en modellen staan
BASE_DIR = "datasets"

# 📷 Open de camera
cap = cv2.VideoCapture(1)
lock = Lock()

## inladen LBPH-modellen

In [3]:
# OpenCV haarcascade gezichtsdetectie
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# **📂 Automatisch alle modellen laden**
models = {}
for person in os.listdir(BASE_DIR):
    model_path = os.path.join(BASE_DIR, person, f"face_model_{person}.yml")
    if os.path.exists(model_path):
        model = cv2.face.LBPHFaceRecognizer_create()
        model.read(model_path)
        models[person] = model
        print(f"✅ Model geladen voor {person}")

if not models:
    print("⚠️ Geen modellen correct geladen. Train eerst een model.")

image_widget = widgets.Image(format='jpeg')
stop_button = widgets.Button(description="Stop")
threshold_slider = widgets.IntSlider(value=100, min=50, max=150, step=1, description="Threshold")
output = widgets.Output()
display(threshold_slider, image_widget, stop_button, output)
running = True

# **🎥 Live gezichtsdetectie en herkenning**
def update_stream():
    global running
    while running:
        with lock:
            cap.grab()
            ret, frame = cap.read()
        
        if not ret:
            print("⚠️ Geen frame ontvangen. Controleer de camera.")
            break

        # **🔍 Converteer naar grijswaarden voor detectie**
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))

        for (x, y, w, h) in faces:
            face_img = gray[y:y+h, x:x+w]
            face_resized = cv2.resize(face_img, (150, 150))  # Formaat voor herkenning

            best_name = "Unknown"
            best_confidence = float("inf")

            # **📌 Haal de actuele threshold op uit de slider**
            confidence_threshold = threshold_slider.value

            # **🔍 Vergelijk gezicht met alle modellen**
            for name, model in models.items():
                label, confidence = model.predict(face_resized)
                print(f"🔍 Model {name}: Confidence {confidence} (Lager is beter)")

                if confidence < best_confidence:
                    best_confidence = confidence
                    best_name = name
            
            # **🟢 Toon groen kader + naam als het onder de threshold zit**
            if best_confidence < confidence_threshold:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, best_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                print(f"✅ Herkend als {best_name} met confidence {best_confidence}")
            else:
                print(f"❌ Gezicht niet zeker genoeg: {best_name} ({best_confidence})")

        # **📸 Toon live beeld in Jupyter**
        _, buffer = cv2.imencode('.jpg', frame)
        image_widget.value = buffer.tobytes()
        
        time.sleep(0.03)

def stop_stream(_):
    global running
    running = False
    with lock:
        cap.release()

stop_button.on_click(stop_stream)

# **🚀 Start de live stream in een aparte thread**
thread = Thread(target=update_stream, daemon=True)
thread.start()


✅ Model geladen voor Arend
✅ Model geladen voor Bobby
✅ Model geladen voor PJ
✅ Model geladen voor Robert
✅ Model geladen voor Unknown


IntSlider(value=100, description='Threshold', max=150, min=50)

Image(value=b'', format='jpeg')

Button(description='Stop', style=ButtonStyle())

Output()

🔍 Model Arend: Confidence 86.08535982283327 (Lager is beter)
🔍 Model Bobby: Confidence 76.77790388666546 (Lager is beter)
🔍 Model PJ: Confidence 92.11366981670712 (Lager is beter)
🔍 Model Robert: Confidence 86.212716238485 (Lager is beter)
🔍 Model Unknown: Confidence 90.7552659149499 (Lager is beter)
✅ Herkend als Bobby met confidence 76.77790388666546
🔍 Model Arend: Confidence 85.14265828682483 (Lager is beter)
🔍 Model Bobby: Confidence 79.04817330667355 (Lager is beter)
🔍 Model PJ: Confidence 91.1745056978693 (Lager is beter)
🔍 Model Robert: Confidence 84.80550024909498 (Lager is beter)
🔍 Model Unknown: Confidence 90.20689538620822 (Lager is beter)
✅ Herkend als Bobby met confidence 79.04817330667355
🔍 Model Arend: Confidence 89.03461094480205 (Lager is beter)
🔍 Model Bobby: Confidence 78.01421187368105 (Lager is beter)
🔍 Model PJ: Confidence 98.50449631686215 (Lager is beter)
🔍 Model Robert: Confidence 93.61304508066665 (Lager is beter)
🔍 Model Unknown: Confidence 96.05849712385678 